<a href="https://colab.research.google.com/github/worldhello23/colab/blob/main/local_Law_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install and Import libraries

In [ ]:
# !pip install -qqq huggingface-hub
# !pip install -qqq pinecone-client
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS
!pip install -qqq llama-cpp-python #Using llama cpp python
# !pip install langchain

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from llama_cpp import Llama
from pinecone import Pinecone
from huggingface_hub import hf_hub_download
from google.colab import userdata

ModuleNotFoundError: No module named 'llama_cpp'

## Embedding model

In [ ]:
model_name_or_path = "CompendiumLabs/bge-large-en-v1.5-gguf"
model_basename = 'bge-large-en-v1.5-f32.gguf'

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename,
    cache_dir= '/content/models' # Directory for the model
)

NameError: name 'hf_hub_download' is not defined

In [ ]:
model = Llama(model_path, embedding=True)

## Pinecone querying

In [ ]:
api_key = userdata.get('pinecone_api')

In [ ]:
pc = Pinecone(api_key=api_key)
index = pc.Index("law")

In [ ]:
query_p = "If I leave the territory of Lithuania, will my Permanent residence become invalid?"
query = model.create_embedding(query_p)
q = query['data'][0]['embedding']


llama_print_timings:        load time =     840.37 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     709.49 ms /    17 tokens (   41.73 ms per token,    23.96 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     839.91 ms /    18 tokens


In [ ]:
response = index.query(
  vector=q,
  top_k=1,
  include_metadata = True,
  namespace = "ns1"
)

In [ ]:
response_t = response['matches'][0]['metadata']['text']

## LLM model

In [ ]:
model_2_name = "TheBloke/stablelm-zephyr-3b-GGUF"#"TheBloke/Llama-2-7B-Chat-GGUF"#"TheBloke/zephyr-7B-beta-GGUF"
model_2base_name = "stablelm-zephyr-3b.Q4_K_M.gguf"#"llama-2-7b-chat.Q4_K_M.gguf" #"zephyr-7b-beta.Q4_K_M.gguf"


In [ ]:
model_path_model = hf_hub_download(
    repo_id=model_2_name,
    filename=model_2base_name,
    cache_dir= '/content/models' # Directory for the model
)

In [ ]:
prompt_template ="<|user|>\
{prompt}<|endoftext|>\
<|assistant|>"

In [ ]:
template = prompt_template
prompt = PromptTemplate.from_template(template)

In [ ]:
response = prompt.format(prompt =f"Based on this {response_t} , answer this {query_p}.")

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
llm = LlamaCpp(
    model_path=model_path_model,
    n_threads = 2,
    temperature=0,
    max_tokens=200,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,
    n_ctx=700
    # Verbose is required to pass to the callback manager

)

llama_model_loader: loaded meta data with 21 key-value pairs and 356 tensors from /content/models/models--TheBloke--stablelm-zephyr-3b-GGUF/snapshots/465310e5eb914190e89f531cc1c348812e27dcea/stablelm-zephyr-3b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = stablelm
llama_model_loader: - kv   1:                               general.name str              = source
llama_model_loader: - kv   2:                    stablelm.context_length u32              = 4096
llama_model_loader: - kv   3:                  stablelm.embedding_length u32              = 2560
llama_model_loader: - kv   4:                       stablelm.block_count u32              = 32
llama_model_loader: - kv   5:               stablelm.feed_forward_length u32              = 6912
llama_model_loader: - kv   6:              stablelm.rope.dimension_

In [ ]:
lllm = Llama(
    model_path=model_path_model,
    n_threads=30,
    n_ctx = 2000# Number of CPU cores
    # n_batch=512       # Batch size, consider your GPU's VRAM
   # Adjust based on your model and GPU VRAM
)

llama_model_loader: loaded meta data with 21 key-value pairs and 356 tensors from /content/models/models--TheBloke--stablelm-zephyr-3b-GGUF/snapshots/465310e5eb914190e89f531cc1c348812e27dcea/stablelm-zephyr-3b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = stablelm
llama_model_loader: - kv   1:                               general.name str              = source
llama_model_loader: - kv   2:                    stablelm.context_length u32              = 4096
llama_model_loader: - kv   3:                  stablelm.embedding_length u32              = 2560
llama_model_loader: - kv   4:                       stablelm.block_count u32              = 32
llama_model_loader: - kv   5:               stablelm.feed_forward_length u32              = 6912
llama_model_loader: - kv   6:              stablelm.rope.dimension_

In [ ]:
# llm.invoke(response)
# resp = lllm(response)['choices'][0]['text']

Llama.generate: prefix-match hit


KeyboardInterrupt: 

## Editing index if needed

Reading the Law File

In [ ]:
p = open('law20233.txt')
data = p.read()

Separate By Articles

In [ ]:
def separate_articles(text):
  """
  This function separates a string containing Lithuanian law text into individual articles
  Args:
      text: The string containing the law text.
  Returns:
      A list of strings, where each string represents a single article.
  """
  articles = []

  current_article = ""

  # Split the text by lines
  lines = text.splitlines()

  for line in lines:
    # Check if the line is an article heading (starts with "Article")
    if line.startswith("Article"):
      # If there's a previous article, add it to the list
      if current_article:
        articles.append(current_article.strip())
      # Start a new article
      current_article = line.strip()
    else:
      # Append non-heading lines to the current article
      current_article += "\n" + line.strip()

  # Add the last article if it exists
  if current_article:
    articles.append(current_article.strip())

  return articles

In [ ]:
data_sep = separate_articles(data)

Embedding the data with model

In [ ]:
# embed = model.embed(data_sep)

Upserting (first time only)

In [ ]:
# formatted_embeddings = []
# for i, embedding in enumerate(embed):
#     embedding_id = str(i)
#     formatted_embedding = {"id": embedding_id, "values": embedding , "metadata": {"text": data_sep[i]}}
#     formatted_embeddings.append(formatted_embedding)
# index.upsert(formatted_embeddings, namespace= "ns1")

Deleting index and namespace

In [ ]:
#  index.delete(delete_all=True, namespace='ns1') #deleting namespace
# pc.delete_index("law") #deleting index